In [1]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
import dill
import numpy as np
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2

In [2]:
import qiskit_nature
qiskit_nature.__version__

'0.7.2'

In [3]:
BL = 0.6

In [4]:
# Parameters
BL = 3.0


### 1 Construct the Hamiltonian

Helper function to construct the Hamiltonian represented as a Pauli decomposition stored in `qubit_op`, while also returning the `nuclear_repulsion_energy` and `core_electron_energy`.

In [5]:
def get_qubit_op(BL):
    molecule = MoleculeInfo(
        symbols=["H", "H"],
        coords=([0.0, 0.0, 0.0], [BL, 0.0, 0.0]),
        multiplicity=1,
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    properties = driver.run()

    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)

    # Get the fermionic Hamiltonian (first element)
    fermionic_op, _ = problem.second_q_ops()

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    # Use Jordan-Wigner (no tapering)
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(fermionic_op)
    # converter = QubitConverter(mapper=mapper)
    # qubit_op = converter.convert(fermionic_op, num_particles=num_particles)

    core_electron_energy = problem.hamiltonian.constants["FreezeCoreTransformer"]
    nuclear_repulsion_energy = problem.nuclear_repulsion_energy

    return (
        qubit_op,
        num_particles,
        num_spatial_orbitals,
        problem,
        mapper,
        nuclear_repulsion_energy,
        core_electron_energy,
    )


In [6]:
(qubit_op, num_particles, num_spatial_orbitals, problem, mapper,nuclear_repulsion_energy, core_electron_energy) = get_qubit_op(BL)

In [7]:
qubit_op

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.72190012+0.j,  0.04516365+0.j,  0.03392839+0.j,  0.04407181+0.j,
  0.04516365+0.j,  0.11784055+0.j,  0.03392839+0.j,  0.1188747 +0.j,
  0.07480289+0.j,  0.07480289+0.j,  0.07480289+0.j,  0.07480289+0.j,
  0.1188747 +0.j,  0.11998245+0.j,  0.04407181+0.j])

In [8]:
# Extract terms and coefficients
pauli_list = list(zip(qubit_op.paulis, qubit_op.coeffs))

# Sort terms by absolute coefficient magnitude (largest first)
sorted_pauli_list = sorted(pauli_list, key=lambda x: abs(x[1]), reverse=True)

# Print sorted terms
# for pauli, coef in sorted_pauli_list:
#     print(f"Pauli: {pauli}, Coefficient: {coef}")

# Save the list
with open("sorted_pauli_list.pkl", "wb") as f:
    dill.dump(sorted_pauli_list, f)

In [9]:
dd = np.real(min(np.linalg.eigvals(qubit_op.to_matrix())))

### 2 Perform VQE

Define a class to perform VQE.

In [10]:
# Define the custom VQE
class CustomVQE:
    def __init__(self, estimator, ansatz, optimizer, qubit_op, dd, NREplusCEE, custom_function, initial_point=None):
        self.estimator = estimator
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.qubit_op = qubit_op
        self.NREplusCEE = NREplusCEE
        self.dd = dd
        self.custom_function = custom_function
        self.initial_point = initial_point  or [0] * ansatz.num_parameters

    def compute_minimum_eigenvalue(self):
        # Define the function to minimize
        def objective_function(params):
            # Evaluate the expectation value of the qubit operator
            expectation = self.estimator.run([self.ansatz], [self.qubit_op], [params]).result().values[0]
            # Evaluate the custom objective function
            return self.custom_function(expectation, params)

        # Use the `minimize` method for optimization
        opt_result = self.optimizer.minimize(
            fun=objective_function,
            x0=self.initial_point
        )

        result = {
            'x': [float(value) for value in opt_result.x],
            'fun': opt_result.fun,
            'dd': self.dd,
            'NREplusCEE': self.NREplusCEE
        }

        
        return result


In [11]:
# l=12 and maxiter = 200 works for LiH and HF

In [12]:
optimizer = SLSQP(maxiter=300)
noiseless_estimator = Estimator(approximation=True)


# Define ansatz
var_form = EfficientSU2(qubit_op.num_qubits, reps=12, entanglement='full', skip_unentangled_qubits=False, parameter_prefix='a')


# Example Custom Objective Function
def custom_objective_function(expectation_value, parameters):
    # penalty_term = 0.1 * sum(param**2 for param in parameters)  # Example regularization
    return expectation_value 


# Create an instance of CustomVQE
custom_vqe = CustomVQE(
    estimator=noiseless_estimator,
    ansatz=var_form,
    optimizer=optimizer,
    qubit_op=qubit_op,
    dd = dd,
    NREplusCEE = nuclear_repulsion_energy + core_electron_energy,
    custom_function=custom_objective_function,
    initial_point = list(np.random.uniform(-np.pi, np.pi, var_form.num_parameters))
)


In [13]:
vqe_result = custom_vqe.compute_minimum_eigenvalue()

In [14]:
vqe_result

{'x': [2.841552761133338,
  0.0966480982350302,
  -2.2422373091246817,
  -0.7083151122136464,
  -1.5550507475713264,
  2.3426751895548272,
  2.7351764499300844,
  -0.14015447963061828,
  -2.363433343608333,
  -1.5856942954959972,
  -0.14714699243307738,
  -1.7441499480880946,
  -3.236137150452624,
  1.0810423105072715,
  1.1479333727685799,
  1.626939412223977,
  2.451972153630861,
  0.5453864998777538,
  -2.6526231093865427,
  -0.5131970737293843,
  -2.786127611835622,
  2.333515696067754,
  -1.8083319158581197,
  1.0459564097910874,
  -0.1253386655082302,
  -1.1206109328001084,
  -0.2489500481801051,
  -0.9715839873568739,
  0.4064080658738802,
  2.528286091439033,
  1.015666802015797,
  -1.679205874625672,
  -1.6792059909896433,
  -2.4018204333183473,
  -3.2588653478777427,
  -1.7263409903938491,
  2.939004420241926,
  -2.1534337636565186,
  -0.20606269784053596,
  1.8164887010089839,
  2.200981626501759,
  -0.26084747243529455,
  -1.305001858272186,
  -2.5593966403672628,
  2.54740

In [15]:
len(vqe_result['x'])

104

In [16]:
print(vqe_result['fun'] - vqe_result['dd'])

0.00048244241125150467


In [17]:
with open("vqe_result.pkl", "wb") as f:
    dill.dump(vqe_result, f)